# Pre-install

#### Install any necessary library / package if not install


In [ ]:
# Uncomment the following lines to install the required packages
# !pip install psycopg2
# !pip install pandas

# Pre-define some data


In [1]:
import psycopg2
import pandas as pd

conn_params = {
    "dbname": "postgres",
    "user": "postgres",
    "password": "12345",
    "host": "localhost",
    "port": "5432",
}

students_data = [
    ("John", "Doe", "Computer Science"),
    ("Jane", "Smith", "Mathematics"),
    ("Emily", "Johnson", "Physics"),
    ("Michael", "Brown", "Engineering"),
    ("Luan", "Tran", "Data Science"),
]

courses_data = [
    ("Introduction to Python", "Computer Science", 2),
    ("Calculus I", "Mathematics", 1),
    ("Physics I", "Physics", 5),
    ("Engineering Design", "Engineering", 3),
]

# Define some necessary functions


### connect to database


In [2]:
def connect():
    """Establishes a connection to the PostgreSQL database and returns the connection and cursor."""
    try:
        conn = psycopg2.connect(**conn_params)
        cur = conn.cursor()
        print("Connection to PostgreSQL established successfully.")
        return conn, cur
    except OperationalError as e:
        print("The error occurred:", e)
        return None, None


def check_connect(conn, cur):
    assert conn is not None, "Database connection failed!"
    print("Database connection is active.")
    assert cur is not None, "Cursor creation failed!"
    print("Cursor created successfully.")
    return True

In [3]:
conn, cur = connect()

Connection to PostgreSQL established successfully.


In [4]:
check_connect(conn, cur)

Database connection is active.
Cursor created successfully.


True

### display retrieved data as a table with some styling


In [5]:
def display_df(df):
    styled_df = (
        df.style.set_properties(
            **{
                "text-align": "center",
                "border": "1px solid black",
                "color": "black",
            }  # Set border and text color
        )
        .set_table_styles(
            [  # Set bold header, background color for header, and black text
                {
                    "selector": "th",
                    "props": [
                        ("font-weight", "bold"),
                        ("background-color", "#f2f2f2"),
                        ("color", "black"),  # Ensure header text is black
                    ],
                }
            ]
        )
        .applymap(
            lambda x: "background-color: #f9f9f9", subset=pd.IndexSlice[::2, :]
        )  # Row background color for even rows
        .applymap(
            lambda x: "background-color: #e0e0e0", subset=pd.IndexSlice[1::2, :]
        )  # Row background color for odd rows
    )
    return styled_df

# Initial Database

### Feeding some initial data into Students and Courses table


In [6]:
def precondition():
    try:
        conn, cur = connect()
        check_connect(conn, cur)
        # If the connection is active, proceed with data insertion
        if conn is not None and cur is not None:
            # Insert data into the Students table
            for student in students_data:
                cur.execute(
                    """
                INSERT INTO Students (first_name, last_name, major)
                VALUES (%s, %s, %s)
                """,
                    student,
                )

            # Insert data into the Courses table
            for course in courses_data:
                cur.execute(
                    """
                INSERT INTO Courses (course_name, department, max_capacity)
                VALUES (%s, %s, %s)
                """,
                    course,
                )

            # Commit the transactions to save the data
            conn.commit()
            print("\nData inserted successfully into Students and Courses tables.")

            # Query and display the inserted data to confirm
            cur.execute("SELECT * FROM Students")
            students_result = cur.fetchall()
            column_names = [desc[0] for desc in cur.description]
            print("\nStudents Table Data:")
            df = pd.DataFrame(students_result, columns=column_names)
            styled_df = display_df(df)
            display(styled_df)

            cur.execute("SELECT * FROM Courses")
            courses_result = cur.fetchall()
            column_names = [desc[0] for desc in cur.description]
            print("\nCourses Table Data:")
            df = pd.DataFrame(
                courses_result,
                columns=column_names,
            )
            styled_df = display_df(df)
            display(styled_df)

    except AssertionError as e:
        print(e)
    except Exception as e:
        print("\nThe error occurred:", e)
    finally:
        # Ensure the cursor and connection are closed if they were created
        if "cur" in locals() and cur is not None:
            cur.close()
        if "conn" in locals() and conn is not None:
            conn.close()
        print("Connection closed.")


precondition()

Connection to PostgreSQL established successfully.
Database connection is active.
Cursor created successfully.

Data inserted successfully into Students and Courses tables.

Students Table Data:


,student_id,first_name,last_name,major
0,1,John,Doe,Computer Science
1,2,Jane,Smith,Mathematics
2,3,Emily,Johnson,Physics
3,4,Michael,Brown,Engineering
4,5,Luan,Tran,Data Science



Courses Table Data:


,course_id,course_name,department,max_capacity,course_capacity
0,1,Introduction to Python,Computer Science,2,0
1,2,Calculus I,Mathematics,1,0
2,3,Physics I,Physics,5,0
3,4,Engineering Design,Engineering,3,0


Connection closed.


# Calling Stored Procedure


In [7]:
def query_stored_procedure_enroll_student(student_id=None, course_id=None):
    # Prompt for student ID and course ID
    if student_id is None:
        student_id = input("Enter the student ID: ")
    if course_id is None:
        course_id = input("Enter the course ID: ")
    try:
        conn, cur = connect()
        check_connect(conn, cur)
        # If the connection is active, proceed with data insertion
        if conn is not None and cur is not None:
            # Enroll the student in the course
            cur.execute(
                """
            CALL enrollstudent(%s, %s)
            """,
                (student_id, course_id),
            )
            conn.commit()
            print("Student enrolled successfully in the course.")

            # Query and display the updated data to confirm
            cur.execute("SELECT * FROM Enrollments")
            enrollments_result = cur.fetchall()
            column_names = [desc[0] for desc in cur.description]
            print("\nEnrollments Table Data:")
            df = pd.DataFrame(
                enrollments_result,
                columns=column_names,
            )

            styled_df = display_df(df)
            display(styled_df)

    except AssertionError as e:
        print(e)
    except Exception as e:
        print("\nThe error occurred:", e)
    finally:
        # Ensure the cursor and connection are closed if they were created
        if "cur" in locals() and cur is not None:
            cur.close()
        if "conn" in locals() and conn is not None:
            conn.close()
        print("Connection closed.\n")

In [14]:
query_stored_procedure_enroll_student()

Enter the student ID: 5
Enter the course ID: 3
Connection to PostgreSQL established successfully.
Database connection is active.
Cursor created successfully.
Student enrolled successfully in the course.

Enrollments Table Data:


,enrollment_id,student_id,course_id,enrollment_date
0,1,5,3,2024-11-08 17:18:15.711728


Connection closed.



In [15]:
query_stored_procedure_enroll_student(1, 3)

Connection to PostgreSQL established successfully.
Database connection is active.
Cursor created successfully.
Student enrolled successfully in the course.

Enrollments Table Data:


,enrollment_id,student_id,course_id,enrollment_date
0,1,5,3,2024-11-08 17:18:15.711728
1,2,1,3,2024-11-08 17:18:20.203718


Connection closed.



# Querrying the Views


In [16]:
def query_views():
    try:
        conn, cur = connect()
        check_connect(conn, cur)
        # If the connection is active, proceed with data insertion
        if conn is not None and cur is not None:
            # Query and display the view data
            cur.execute("SELECT * FROM StudentCourseView")
            student_courses_result = cur.fetchall()
            column_names = [desc[0] for desc in cur.description]
            print("\nStudentCourses View Data:")
            df = pd.DataFrame(
                student_courses_result,
                columns=column_names,
            )
            styled_df = display_df(df)
            display(styled_df)

            cur.execute("SELECT * FROM CourseCapacityView")
            course_capacity_result = cur.fetchall()
            column_names = [desc[0] for desc in cur.description]
            print("\nCourseCapacity View Data:")
            df = pd.DataFrame(
                course_capacity_result,
                columns=column_names,
            )
            styled_df = display_df(df)
            display(styled_df)

    except AssertionError as e:
        print(e)
    except Exception as e:
        print("\nThe error occurred:", e)
    finally:
        # Ensure the cursor and connection are closed if they were created
        if "cur" in locals() and cur is not None:
            cur.close()
        if "conn" in locals() and conn is not None:
            conn.close()
        print("Connection closed.\n")


query_views()

Connection to PostgreSQL established successfully.
Database connection is active.
Cursor created successfully.

StudentCourses View Data:


,student_id,student_name,course_name,enrollment_date
0,5,Luan Tran,Physics I,2024-11-08 17:18:15.711728
1,1,John Doe,Physics I,2024-11-08 17:18:20.203718



CourseCapacity View Data:


,course_id,course_name,department,current_enrollment,remaining_capacity
0,4,Engineering Design,Engineering,0,3
1,2,Calculus I,Mathematics,0,1
2,1,Introduction to Python,Computer Science,0,2
3,3,Physics I,Physics,2,3


Connection closed.



# Triggering Events


In [17]:
def trigger_event(student_id, course_id, action="enroll"):
    try:
        conn, cur = connect()
        check_connect(conn, cur)
        # If the connection is active, proceed with data insertion
        if conn is not None and cur is not None:
            # Enroll or drop the student from the course based on the action
            if action == "enroll":
                cur.execute("""CALL enrollstudent(%s, %s)""", (student_id, course_id))
                print("Student enrolled successfully in the course.")
            elif action == "drop":
                # Check if the record exists before attempting to delete
                cur.execute(
                    """
                    SELECT 1 FROM Enrollments
                    WHERE student_id = %s AND course_id = %s
                    """,
                    (student_id, course_id),
                )
                if cur.fetchone():
                    cur.execute(
                        """
                        DELETE FROM Enrollments
                        WHERE student_id = %s AND course_id = %s
                        """,
                        (student_id, course_id),
                    )
                    print("Student dropped successfully from the course.")
                else:
                    raise Exception("The student is not enrolled in the course.")

            conn.commit()

            # Query and display the updated data to confirm
            cur.execute("SELECT * FROM EnrollmentLog")
            enrollment_log_result = cur.fetchall()
            column_names = [desc[0] for desc in cur.description]
            print("Check the updated enrollment log after the event:")
            print("\nEnrollmentLog Table Data:")
            df = pd.DataFrame(
                enrollment_log_result,
                columns=column_names,
            )
            styled_df = display_df(df)
            display(styled_df)

            cur.execute("SELECT * FROM Courses")
            courses_result = cur.fetchall()
            column_names = [desc[0] for desc in cur.description]
            print("Check the updated course capacity after the event:")
            print("\nCourses Table Data:")
            df = pd.DataFrame(
                courses_result,
                columns=column_names,
            )
            styled_df = display_df(df)
            display(styled_df)

    except AssertionError as e:
        print(e)
    except Exception as e:
        print("\nThe error occurred:", e)
    finally:
        # Ensure the cursor and connection are closed if they were created
        if "cur" in locals() and cur is not None:
            cur.close()
        if "conn" in locals() and conn is not None:
            conn.close()
        print("Connection closed.\n")

In [18]:
trigger_event(3, 3, "enroll")

Connection to PostgreSQL established successfully.
Database connection is active.
Cursor created successfully.
Student enrolled successfully in the course.
Check the updated enrollment log after the event:

EnrollmentLog Table Data:


,log_id,student_id,course_id,action,action_timestamp
0,1,5,3,enroll,2024-11-08 17:18:15.711728
1,2,1,3,enroll,2024-11-08 17:18:20.203718
2,3,3,3,enroll,2024-11-08 17:18:29.702561


Check the updated course capacity after the event:

Courses Table Data:


,course_id,course_name,department,max_capacity,course_capacity
0,1,Introduction to Python,Computer Science,2,0
1,2,Calculus I,Mathematics,1,0
2,4,Engineering Design,Engineering,3,0
3,3,Physics I,Physics,5,3


Connection closed.



In [19]:
trigger_event(3, 3, "drop")

Connection to PostgreSQL established successfully.
Database connection is active.
Cursor created successfully.
Student dropped successfully from the course.
Check the updated enrollment log after the event:

EnrollmentLog Table Data:


,log_id,student_id,course_id,action,action_timestamp
0,1,5,3,enroll,2024-11-08 17:18:15.711728
1,2,1,3,enroll,2024-11-08 17:18:20.203718
2,3,3,3,enroll,2024-11-08 17:18:29.702561
3,4,3,3,drop,2024-11-08 17:18:30.834361


Check the updated course capacity after the event:

Courses Table Data:


,course_id,course_name,department,max_capacity,course_capacity
0,1,Introduction to Python,Computer Science,2,0
1,2,Calculus I,Mathematics,1,0
2,4,Engineering Design,Engineering,3,0
3,3,Physics I,Physics,5,2


Connection closed.



# Test Case


### Our current database is showned below


In [20]:
def current_database():
    try:
        conn, cur = connect()
        check_connect(conn, cur)
        # If the connection is active, proceed with data insertion
        if conn is not None and cur is not None:
            # Query and display the inserted data to confirm

            # Students Table Data
            cur.execute("SELECT * FROM Students")
            students_result = cur.fetchall()
            column_names = [desc[0] for desc in cur.description]
            print("\nStudents Table Data:")
            df = pd.DataFrame(students_result, columns=column_names)
            styled_df = display_df(df)
            display(styled_df)

            # Courses Table Data
            cur.execute("SELECT * FROM Courses")
            courses_result = cur.fetchall()
            column_names = [desc[0] for desc in cur.description]
            print("\nCourses Table Data:")
            df = pd.DataFrame(
                courses_result,
                columns=column_names,
            )
            styled_df = display_df(df)
            display(styled_df)

    except AssertionError as e:
        print(e)
    except Exception as e:
        print("The error occurred:", e)
    finally:
        # Ensure the cursor and connection are closed if they were created
        if "cur" in locals() and cur is not None:
            cur.close()
        if "conn" in locals() and conn is not None:
            conn.close()
        print("Connection closed.")


current_database()

Connection to PostgreSQL established successfully.
Database connection is active.
Cursor created successfully.

Students Table Data:


,student_id,first_name,last_name,major
0,1,John,Doe,Computer Science
1,2,Jane,Smith,Mathematics
2,3,Emily,Johnson,Physics
3,4,Michael,Brown,Engineering
4,5,Luan,Tran,Data Science



Courses Table Data:


,course_id,course_name,department,max_capacity,course_capacity
0,1,Introduction to Python,Computer Science,2,0
1,2,Calculus I,Mathematics,1,0
2,4,Engineering Design,Engineering,3,0
3,3,Physics I,Physics,5,2


Connection closed.


### current tracking table


In [21]:
def track_database():
    try:
        conn, cur = connect()
        check_connect(conn, cur)
        # If the connection is active, proceed with data insertion
        if conn is not None and cur is not None:
            # Query and display the inserted data to confirm

            # Enrollments Table Data
            cur.execute("SELECT * FROM Enrollments")
            enrollments_result = cur.fetchall()
            column_names = [desc[0] for desc in cur.description]
            print("\nEnrollments Table Data:")
            df = pd.DataFrame(enrollments_result, columns=column_names)
            styled_df = display_df(df)
            display(styled_df)

            # Courses Table Data
            cur.execute("SELECT * FROM Courses")
            courses_result = cur.fetchall()
            column_names = [desc[0] for desc in cur.description]
            print("\nCourses Table Data:")
            df = pd.DataFrame(
                courses_result,
                columns=column_names,
            )
            styled_df = display_df(df)
            display(styled_df)

            # EnrollmentLog Table Data
            cur.execute("SELECT * FROM EnrollmentLog")
            enrollment_log_result = cur.fetchall()
            column_names = [desc[0] for desc in cur.description]
            print("\nEnrollmentLog Table Data:")
            df = pd.DataFrame(
                enrollment_log_result,
                columns=column_names,
            )
            styled_df = display_df(df)
            display(styled_df)

    except AssertionError as e:
        print(e)
    except Exception as e:
        print("The error occurred:", e)
    finally:
        # Ensure the cursor and connection are closed if they were created
        if "cur" in locals() and cur is not None:
            cur.close()
        if "conn" in locals() and conn is not None:
            conn.close()
        print("Connection closed.")


track_database()

Connection to PostgreSQL established successfully.
Database connection is active.
Cursor created successfully.

Enrollments Table Data:


,enrollment_id,student_id,course_id,enrollment_date
0,1,5,3,2024-11-08 17:18:15.711728
1,2,1,3,2024-11-08 17:18:20.203718



Courses Table Data:


,course_id,course_name,department,max_capacity,course_capacity
0,1,Introduction to Python,Computer Science,2,0
1,2,Calculus I,Mathematics,1,0
2,4,Engineering Design,Engineering,3,0
3,3,Physics I,Physics,5,2



EnrollmentLog Table Data:


,log_id,student_id,course_id,action,action_timestamp
0,1,5,3,enroll,2024-11-08 17:18:15.711728
1,2,1,3,enroll,2024-11-08 17:18:20.203718
2,3,3,3,enroll,2024-11-08 17:18:29.702561
3,4,3,3,drop,2024-11-08 17:18:30.834361


Connection closed.


### Test case 1 : Course is full


enrollStudent(5, 2) make the course full


In [22]:
query_stored_procedure_enroll_student(5, 2)

Connection to PostgreSQL established successfully.
Database connection is active.
Cursor created successfully.
Student enrolled successfully in the course.

Enrollments Table Data:


,enrollment_id,student_id,course_id,enrollment_date
0,1,5,3,2024-11-08 17:18:15.711728
1,2,1,3,2024-11-08 17:18:20.203718
2,4,5,2,2024-11-08 17:18:42.180006


Connection closed.



Try doing enrollStudent(4, 2)


In [23]:
query_stored_procedure_enroll_student(4, 2)

Connection to PostgreSQL established successfully.
Database connection is active.
Cursor created successfully.

The error occurred: Course is full
CONTEXT:  PL/pgSQL function enrollstudent(integer,integer) line 40 at RAISE

Connection closed.



Checking any changes in database


In [24]:
track_database()

Connection to PostgreSQL established successfully.
Database connection is active.
Cursor created successfully.

Enrollments Table Data:


,enrollment_id,student_id,course_id,enrollment_date
0,1,5,3,2024-11-08 17:18:15.711728
1,2,1,3,2024-11-08 17:18:20.203718
2,4,5,2,2024-11-08 17:18:42.180006



Courses Table Data:


,course_id,course_name,department,max_capacity,course_capacity
0,1,Introduction to Python,Computer Science,2,0
1,4,Engineering Design,Engineering,3,0
2,3,Physics I,Physics,5,2
3,2,Calculus I,Mathematics,1,1



EnrollmentLog Table Data:


,log_id,student_id,course_id,action,action_timestamp
0,1,5,3,enroll,2024-11-08 17:18:15.711728
1,2,1,3,enroll,2024-11-08 17:18:20.203718
2,3,3,3,enroll,2024-11-08 17:18:29.702561
3,4,3,3,drop,2024-11-08 17:18:30.834361
4,5,5,2,enroll,2024-11-08 17:18:42.180006


Connection closed.


### Test case 2 : Course does not exist


In [25]:
query_stored_procedure_enroll_student(1, 10)

Connection to PostgreSQL established successfully.
Database connection is active.
Cursor created successfully.

The error occurred: Course does not exist
CONTEXT:  PL/pgSQL function enrollstudent(integer,integer) line 23 at RAISE

Connection closed.



Checking any changes in database


In [26]:
track_database()

Connection to PostgreSQL established successfully.
Database connection is active.
Cursor created successfully.

Enrollments Table Data:


,enrollment_id,student_id,course_id,enrollment_date
0,1,5,3,2024-11-08 17:18:15.711728
1,2,1,3,2024-11-08 17:18:20.203718
2,4,5,2,2024-11-08 17:18:42.180006



Courses Table Data:


,course_id,course_name,department,max_capacity,course_capacity
0,1,Introduction to Python,Computer Science,2,0
1,4,Engineering Design,Engineering,3,0
2,3,Physics I,Physics,5,2
3,2,Calculus I,Mathematics,1,1



EnrollmentLog Table Data:


,log_id,student_id,course_id,action,action_timestamp
0,1,5,3,enroll,2024-11-08 17:18:15.711728
1,2,1,3,enroll,2024-11-08 17:18:20.203718
2,3,3,3,enroll,2024-11-08 17:18:29.702561
3,4,3,3,drop,2024-11-08 17:18:30.834361
4,5,5,2,enroll,2024-11-08 17:18:42.180006


Connection closed.


### Test case 3 : Student does not exist


In [27]:
query_stored_procedure_enroll_student(6, 1)

Connection to PostgreSQL established successfully.
Database connection is active.
Cursor created successfully.

The error occurred: Student does not exist
CONTEXT:  PL/pgSQL function enrollstudent(integer,integer) line 14 at RAISE

Connection closed.



Checking any changes in database


In [28]:
track_database()

Connection to PostgreSQL established successfully.
Database connection is active.
Cursor created successfully.

Enrollments Table Data:


,enrollment_id,student_id,course_id,enrollment_date
0,1,5,3,2024-11-08 17:18:15.711728
1,2,1,3,2024-11-08 17:18:20.203718
2,4,5,2,2024-11-08 17:18:42.180006



Courses Table Data:


,course_id,course_name,department,max_capacity,course_capacity
0,1,Introduction to Python,Computer Science,2,0
1,4,Engineering Design,Engineering,3,0
2,3,Physics I,Physics,5,2
3,2,Calculus I,Mathematics,1,1



EnrollmentLog Table Data:


,log_id,student_id,course_id,action,action_timestamp
0,1,5,3,enroll,2024-11-08 17:18:15.711728
1,2,1,3,enroll,2024-11-08 17:18:20.203718
2,3,3,3,enroll,2024-11-08 17:18:29.702561
3,4,3,3,drop,2024-11-08 17:18:30.834361
4,5,5,2,enroll,2024-11-08 17:18:42.180006


Connection closed.


#### Test case 4: Student already enrolled


Current Tables


In [29]:
track_database()

Connection to PostgreSQL established successfully.
Database connection is active.
Cursor created successfully.

Enrollments Table Data:


,enrollment_id,student_id,course_id,enrollment_date
0,1,5,3,2024-11-08 17:18:15.711728
1,2,1,3,2024-11-08 17:18:20.203718
2,4,5,2,2024-11-08 17:18:42.180006



Courses Table Data:


,course_id,course_name,department,max_capacity,course_capacity
0,1,Introduction to Python,Computer Science,2,0
1,4,Engineering Design,Engineering,3,0
2,3,Physics I,Physics,5,2
3,2,Calculus I,Mathematics,1,1



EnrollmentLog Table Data:


,log_id,student_id,course_id,action,action_timestamp
0,1,5,3,enroll,2024-11-08 17:18:15.711728
1,2,1,3,enroll,2024-11-08 17:18:20.203718
2,3,3,3,enroll,2024-11-08 17:18:29.702561
3,4,3,3,drop,2024-11-08 17:18:30.834361
4,5,5,2,enroll,2024-11-08 17:18:42.180006


Connection closed.


Try doing EnrollStudent (5, 3)


In [30]:
query_stored_procedure_enroll_student(5, 3)

Connection to PostgreSQL established successfully.
Database connection is active.
Cursor created successfully.

The error occurred: Student is already enrolled in this course
CONTEXT:  PL/pgSQL function enrollstudent(integer,integer) line 51 at RAISE

Connection closed.



After Operation


In [31]:
track_database()

Connection to PostgreSQL established successfully.
Database connection is active.
Cursor created successfully.

Enrollments Table Data:


,enrollment_id,student_id,course_id,enrollment_date
0,1,5,3,2024-11-08 17:18:15.711728
1,2,1,3,2024-11-08 17:18:20.203718
2,4,5,2,2024-11-08 17:18:42.180006



Courses Table Data:


,course_id,course_name,department,max_capacity,course_capacity
0,1,Introduction to Python,Computer Science,2,0
1,4,Engineering Design,Engineering,3,0
2,3,Physics I,Physics,5,2
3,2,Calculus I,Mathematics,1,1



EnrollmentLog Table Data:


,log_id,student_id,course_id,action,action_timestamp
0,1,5,3,enroll,2024-11-08 17:18:15.711728
1,2,1,3,enroll,2024-11-08 17:18:20.203718
2,3,3,3,enroll,2024-11-08 17:18:29.702561
3,4,3,3,drop,2024-11-08 17:18:30.834361
4,5,5,2,enroll,2024-11-08 17:18:42.180006


Connection closed.


### Test case 5 : Trigger Error when Deleting a Non-Existing Enrollment Record and Verify EnrollmentLog


Current tables


In [32]:
track_database()

Connection to PostgreSQL established successfully.
Database connection is active.
Cursor created successfully.

Enrollments Table Data:


,enrollment_id,student_id,course_id,enrollment_date
0,1,5,3,2024-11-08 17:18:15.711728
1,2,1,3,2024-11-08 17:18:20.203718
2,4,5,2,2024-11-08 17:18:42.180006



Courses Table Data:


,course_id,course_name,department,max_capacity,course_capacity
0,1,Introduction to Python,Computer Science,2,0
1,4,Engineering Design,Engineering,3,0
2,3,Physics I,Physics,5,2
3,2,Calculus I,Mathematics,1,1



EnrollmentLog Table Data:


,log_id,student_id,course_id,action,action_timestamp
0,1,5,3,enroll,2024-11-08 17:18:15.711728
1,2,1,3,enroll,2024-11-08 17:18:20.203718
2,3,3,3,enroll,2024-11-08 17:18:29.702561
3,4,3,3,drop,2024-11-08 17:18:30.834361
4,5,5,2,enroll,2024-11-08 17:18:42.180006


Connection closed.


Try dropping an enrollment


In [33]:
trigger_event(1, 1, "drop")

Connection to PostgreSQL established successfully.
Database connection is active.
Cursor created successfully.

The error occurred: The student is not enrolled in the course.
Connection closed.



Try enrolling again


In [34]:
trigger_event(1, 1, "enroll")

Connection to PostgreSQL established successfully.
Database connection is active.
Cursor created successfully.
Student enrolled successfully in the course.
Check the updated enrollment log after the event:

EnrollmentLog Table Data:


,log_id,student_id,course_id,action,action_timestamp
0,1,5,3,enroll,2024-11-08 17:18:15.711728
1,2,1,3,enroll,2024-11-08 17:18:20.203718
2,3,3,3,enroll,2024-11-08 17:18:29.702561
3,4,3,3,drop,2024-11-08 17:18:30.834361
4,5,5,2,enroll,2024-11-08 17:18:42.180006
5,6,1,1,enroll,2024-11-08 17:19:09.983004


Check the updated course capacity after the event:

Courses Table Data:


,course_id,course_name,department,max_capacity,course_capacity
0,4,Engineering Design,Engineering,3,0
1,3,Physics I,Physics,5,2
2,2,Calculus I,Mathematics,1,1
3,1,Introduction to Python,Computer Science,2,1


Connection closed.



Checking any changes in database


In [35]:
track_database()

Connection to PostgreSQL established successfully.
Database connection is active.
Cursor created successfully.

Enrollments Table Data:


,enrollment_id,student_id,course_id,enrollment_date
0,1,5,3,2024-11-08 17:18:15.711728
1,2,1,3,2024-11-08 17:18:20.203718
2,4,5,2,2024-11-08 17:18:42.180006
3,5,1,1,2024-11-08 17:19:09.983004



Courses Table Data:


,course_id,course_name,department,max_capacity,course_capacity
0,4,Engineering Design,Engineering,3,0
1,3,Physics I,Physics,5,2
2,2,Calculus I,Mathematics,1,1
3,1,Introduction to Python,Computer Science,2,1



EnrollmentLog Table Data:


,log_id,student_id,course_id,action,action_timestamp
0,1,5,3,enroll,2024-11-08 17:18:15.711728
1,2,1,3,enroll,2024-11-08 17:18:20.203718
2,3,3,3,enroll,2024-11-08 17:18:29.702561
3,4,3,3,drop,2024-11-08 17:18:30.834361
4,5,5,2,enroll,2024-11-08 17:18:42.180006
5,6,1,1,enroll,2024-11-08 17:19:09.983004


Connection closed.
